# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [1]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [2]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  return [-3*a**2-0.5*a**(-0.5), 2.5*b**1.5+3*cos(3*b), c**-2] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of 
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [9]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
# (f(x+h)-f(x))/h
# In our case:
a1, b1, c1 = 2,3,4
h=0.000001
y1 = f(a1,b1,c1) 
dfda = (f(a1+h, b1, c1)-y1)/h
dfdb = (f(a1, b1+h, c1)-y1)/h
dfdc = (f(a1, b1, c1+h)-y1)/h


numerical_grad = [dfda, dfdb, dfdc]
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


In [10]:
# there is an alternative formula that provides a much better numerical 
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
# (f(x+h)-f(x-h))/2h -->  

a1, b1, c1 = 2,3,4
h=0.000001
dfda = (f(a1+h, b1, c1)-f(a1-h, b1, c1))/(2*h)
dfdb = (f(a1, b1+h, c1)-f(a1, b1-h, c1))/(2*h)
dfdc = (f(a1, b1, c1+h)-f(a1, b1, c1-h))/(2*h)

numerical_grad2 = [dfda, dfdb, dfdc]

# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553391353245
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027401572
OK for dim 2: expected 0.0625, yours returns 0.06250000028629188


## section 2: support for softmax

In [84]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    return out
  
  # ------
  # re-implement all the other functions needed for the exercises below
  # your code here
  # TODO
  # ------

  # multiplication
  
  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other) # makes other a Value(other) if it isn't
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad 
      other.grad += self.data * out.grad
      # say L = d * f, dL/dd = f.
      # And d = c + e, what's dL/dc (so we're at node c for this question!!!)
      # Chain rule: dL/dc = dL/dd * dd/dc !!!! Omg!! out.grad * other.data (idea from dL/dd = f)!!!!!!!!!!1
      # THAT'S BACKPROPAGATION BOI. THAT'S IT. THAT'S THE CRUX OF IT.
    out._backward = _backward
    return out

  def __pow__(self, k):
    assert isinstance(k, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**k, (self,), f'**{k}')

    def _backward():
        self.grad += k * (self.data ** (k - 1)) * out.grad
    out._backward = _backward

    return out
  

  def __truediv__(self, other):
    return self * other**-1

  def __radd__(self, other): # other + self
    return self + other
  def __rmul__(self, other): # other * self
    return self * other

    
  def __neg__(self): # -self
    return self * -1 

  def __sub__(self, other): # self - other
    return self + (-other)


  # log
  def log(self):
    out = Value(log(self.data), (self,), 'log')

    def _backward(): 
      self.grad += out.grad/self.data
    out._backward = _backward
    return out
    

  #exp
  def exp(self):
    exped_up = exp(self.data)
    out = Value(exped_up, (self, ), 'exp')

    def _backward():
      self.grad += exped_up * out.grad
    out._backward = _backward
    return out 


  def backward(self): # exactly as in video  
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [68]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  # AAh, get it? It's a loss function because it's a derivable function that maps R -> Positive.
  # Well, softmax is like gigachad generalised logistic function -- see wiki.
  # "Converts K real numbers into probability distribution of K possible outcomes"
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts] # Prob distr. of thingies essentially, adds to 1. 
  return out

In [91]:
# Testing if it works or not

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
print(probs)
loss = -probs[3].log() # dim 3 acts as the label for this input example
print(f"loss: {loss}")
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


[Value(data=0.04177257051535045), Value(data=0.839024507462532), Value(data=0.00565330266221633), Value(data=0.11354961935990122)]
loss: Value(data=2.1755153626167147)
2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [100]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch

## Stuff from exercises
import torch
logits = torch.Tensor([0.0, 3.0, -2.0, 1.0]); logits.requires_grad = True
probs = torch.softmax(logits, dim=0)

print(probs)

loss = -probs[3].log()
loss.backward()
print(logits.grad)

# Test if right -- this is a bit different; we go logits.grad[dim] instead of logits[dim].grad as Value is different to tensor behaviour
ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits.grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits.grad[dim]}")

tensor([0.0418, 0.8390, 0.0057, 0.1135], grad_fn=<SoftmaxBackward0>)
tensor([ 0.0418,  0.8390,  0.0057, -0.8865])
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772566735744476
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390244841575623
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302650898695
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864504098892212
